In [3]:
import random
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import joblib

# Generowanie danych
branches = ['Branch1', 'Branch2', 'Branch3', 'Branch4', 'Branch5']
data = []

for _ in range(100):  # 100 odczytów dla każdej gałęzi
    for branch in branches:
        energy_usage = round(random.uniform(50.0, 500.0), 2)
        timestamp = str(datetime.utcnow())
        data.append({'branch': branch, 'energy_usage': energy_usage, 'timestamp': timestamp})

df = pd.DataFrame(data)

# Wyświetlenie przykładowych danych
print(df.head())

    branch  energy_usage                   timestamp
0  Branch1        293.49  2024-05-27 19:06:47.413614
1  Branch2        253.62  2024-05-27 19:06:47.413650
2  Branch3        405.98  2024-05-27 19:06:47.413660
3  Branch4        210.59  2024-05-27 19:06:47.413667
4  Branch5        293.97  2024-05-27 19:06:47.413677


In [4]:
# Grupowanie danych po gałęziach
branch_data = df.groupby('branch')

models = {}
for branch, group in branch_data:
    clf = IsolationForest(n_estimators=100, contamination=0.1)
    clf.fit(group['energy_usage'].values.reshape(-1, 1))
    models[branch] = clf
    joblib.dump(clf, f'isolation_forest_{branch}.joblib')  # Zapisywanie modelu

    print(f'Model for {branch} trained and saved.')

Model for Branch1 trained and saved.
Model for Branch2 trained and saved.
Model for Branch3 trained and saved.
Model for Branch4 trained and saved.
Model for Branch5 trained and saved.


In [6]:
# Załadowanie i testowanie modelu
for branch in branches:
    model = joblib.load(f'isolation_forest_{branch}.joblib')
    # Przykładowe dane do testowania
    test_data = np.array([random.uniform(50.0, 500.0) for _ in range(10)]).reshape(-1, 1)
    preds = model.predict(test_data)
    print(f"Predictions for {branch}: {preds}")

Predictions for Branch1: [ 1 -1 -1  1 -1  1  1  1  1  1]
Predictions for Branch2: [ 1  1  1  1  1  1 -1  1  1  1]
Predictions for Branch3: [-1  1  1  1  1  1  1  1  1 -1]
Predictions for Branch4: [ 1  1  1  1 -1  1  1  1 -1  1]
Predictions for Branch5: [ 1  1 -1  1  1  1  1  1  1  1]
